[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anytko/ecospat/blob/main/docs/examples/individual_persistence.ipynb)

In [ ]:
# Uncomment below to run in Google Collab
# pip install ecospat

## Categorizing and visualizing individual persistence

In [ ]:
import ecospat.ecospat as ecospat_full
from ecospat.stand_alone_functions import (
    process_species_historical_range,
    analyze_species_distribution,
    analyze_northward_shift,
    calculate_rate_of_change_first_last,
    merge_category_dataframes,
    prepare_gdf_for_rasterization,
    cat_int_mapping,
    rasterize_multiband_gdf_match,
    compute_propagule_pressure_range,
    compute_individual_persistence,
)

### Historic range edges

In [ ]:
hist_pipeline = ecospat_full.Map()
hist_range = process_species_historical_range(
    new_map=hist_pipeline, species_name="Populus angustifolia"
)

### Modern range edges and historical population data

In [ ]:
classified_modern, classified_historic = analyze_species_distribution(
    "Populus angustifolia", record_limit=1000, continent="north_america"
)

### Northward shift

In [ ]:
northward_rate_df = analyze_northward_shift(
    gdf_hist=hist_range,
    gdf_new=classified_modern,
    species_name="Populus angustifolia",
)
northward_rate_df = northward_rate_df[
    northward_rate_df["category"].isin(["leading", "core", "trailing"])
]

northward_rate_df["category"] = northward_rate_df["category"].str.title()

### Population density change

In [ ]:
change = calculate_rate_of_change_first_last(
    classified_historic, classified_modern, "Populus angustifolia", custom_end_year=2025
)


change = change[change["collapsed_category"].isin(["leading", "core", "trailing"])]
change = change.rename(
    columns={
        "collapsed_category": "Category",
        "rate_of_change_first_last": "Rate of Change",
        "start_time_period": "Start Years",
        "end_time_period": "End Years",
    }
)


change["Category"] = change["Category"].str.title()

### Prepare data for rasterization

In [ ]:
merged = merge_category_dataframes(northward_rate_df, change)

preped_gdf = prepare_gdf_for_rasterization(classified_modern, merged)

preped_gdf_new = cat_int_mapping(preped_gdf)

preped_gdf_new.head()

In [ ]:
value_columns = [
    "density",
    "northward_rate_km_per_year",
    "Rate of Change",
    "category_int",
]
raster_show, transform, show_bounds = rasterize_multiband_gdf_match(
    preped_gdf_new, value_columns
)

### Rasterize data and compute propagule pressure

In [ ]:
pressure_show = compute_propagule_pressure_range(raster_show)

### Compute individual persistence

In [ ]:
points = compute_individual_persistence(
    points_gdf=classified_modern,
    raster_stack_arrays=raster_show,
    propagule_array=pressure_show,
    baseline_death=0.10,
    transform=transform,
)
points.head()

In [ ]:
point_map_new = ecospat_full.Map()
point_map_new.add_basemap("GBIF.Classic")
point_map_new.add_point_data(points, use_gradient=True)
point_map_new